In [1]:
import json

import numpy as np
import pandas as pd
from toolz.curried import *
import missingno
from matplotlib import pyplot as plt

In [2]:
def load_ranking_df():
    filename = '../data/rankings.csv'
    ranking_df = pd.read_csv(filename)
    clean_rankings = ranking_df.drop(columns=['Unnamed: 0'])
    clean_rankings = clean_rankings.drop(0)
    return clean_rankings

def load_athlete_df():
    # When loading, you may want to change the years to ints (future porter: I don't know why I put this here...)
    filename = '../data/athlete_data.csv'
    athlete_df = pd.read_csv(filename)
    clean_athletes = athlete_df.set_index('id')
    clean_athletes.index = clean_athletes.index.rename('ID')
    clean_athletes = clean_athletes.drop(columns='age')

    bad_ages_mask = ~clean_athletes['birth_year'].isin(np.arange(1990-80, 2020-3))
    bad_ages = clean_athletes[bad_ages_mask]['birth_year'].unique()
    bad_ages = bad_ages[1:] # Take the nan out
    clean_athletes.loc[clean_athletes['birth_year'].isin(bad_ages)] = np.nan

    clean_athletes['height'] = clean_athletes['height'].str.slice(0,-3)
    clean_athletes['height'] = clean_athletes['height'].astype(float)
    clean_athletes['weight'] = clean_athletes['weight'].str.slice(0,-2)
    clean_athletes['weight'] = clean_athletes['weight'].astype(float)
    clean_athletes = clean_athletes.rename({'first_name': 'First Name', 'last_name': 'Last Name'}, axis=1)
    
    # Get rid of really short and light athletes (probably kids)
#     clean_athletes.loc[clean_athletes['height'] < 100] = np.nan
#     clean_athletes.loc[clean_athletes['weight'] < 20] = np.nan
    
    return clean_athletes

events = None
def load_event_df():
    if events is None:
        with open('../data/athlete_comps.json') as f:
            data = json.loads(f.read())
            athlete_comp_result = {athlete_id: merge(*[{comp['comp']: comp['result']} for comp in comps]) for athlete_id, comps in data.items()}
            event_df = pd.read_json(json.dumps(athlete_comp_result), orient='index')
        return event_df
    return events

In [3]:
worldcup = pd.read_csv('../data/comp_data_ifsc_worldcup.csv')
rankings = load_ranking_df()
athletes = load_athlete_df()
events = load_event_df()
merged = pd.read_pickle('merged.pkl')
merged = merged.drop(columns='birth_year')

In [4]:
merged[:50]

,ID,Rank,Last Name,First Name,Points,Event,Gender,Year,country,height,weight,age
0,8372,1,Ondra,Adam,300.0,lead,MEN,2019,CZE,185.0,67.0,26.0
1,8372,2,Ondra,Adam,335.0,boulder,MEN,2019,CZE,185.0,67.0,26.0
2,8372,31,Ondra,Adam,55.0,lead,MEN,2018,CZE,185.0,67.0,25.0
3,8372,28,Ondra,Adam,80.0,lead,MEN,2017,CZE,185.0,67.0,24.0
4,8372,24,Ondra,Adam,51.0,lead,MEN,2016,CZE,185.0,67.0,23.0
5,8372,1,Ondra,Adam,458.0,lead,MEN,2015,CZE,185.0,67.0,22.0
6,8372,3,Ondra,Adam,259.0,boulder,MEN,2015,CZE,185.0,67.0,22.0
7,8372,3,Ondra,Adam,428.0,lead,MEN,2014,CZE,185.0,67.0,21.0
8,8372,24,Ondra,Adam,80.0,boulder,MEN,2014,CZE,185.0,67.0,21.0
9,8372,11,Ondra,Adam,180.0,lead,MEN,2013,CZE,185.0,67.0,20.0


In [5]:
men_lead_count_dfs = {}
discipline = 'lead'
for year in range(2019, 1990, -1):
    filepath = '../data/worldcup_data.csv'
    df = pd.read_csv(filepath, index_col=0)
    df = df.drop(columns=['rank', 'title', 'date', 'comp_id', 'cat_id'])
    year_df = df[(df['year']==year) & (df['type']==discipline)]
    counts = year_df['athlete_id'].value_counts()
    men_lead_count_dfs[year] = counts

In [6]:
top_athletes = merged['ID'].unique()

# Point dropping started in 2003. Standardized after 2007?

# ml suffix is for 'men' 'lead'
merged_ml = merged[(merged['Event']=='lead') & (merged['Gender']=='MEN')].copy()
merged_ml = merged_ml.drop(columns=['Event', 'Gender'])
merged_ml = merged_ml.reset_index(drop=True)

all_counts = []
for i, row in merged_ml.iterrows():
    year = row['Year']
    ID = row['ID']
    count_df = men_lead_count_dfs[year]
    all_counts.append(count_df[ID])
    
merged_ml['event_count'] = all_counts
merged_ml['avg_points'] = merged_ml['Points'] / merged_ml['event_count']

K = 7 # num of lags
col_names = [f't-{k+1}' for k in range(K)]
lag_df = pd.DataFrame(columns=col_names)
# print(lag_df)
for i, ID in enumerate(top_athletes):
    athlete = merged_ml[merged_ml['ID']==ID].copy()
    lag_list = [athlete['avg_points'].shift(-(k+1)) for k in range(K)]
    lags = pd.concat(lag_list, axis=1)
    lags.columns = col_names
    lag_df = pd.concat([lag_df, lags], axis=0)

merged_ml = pd.concat([merged_ml, lag_df], axis=1)
merged_ml.head(30) # This one goes by athlete, rather than by year

,ID,Rank,Last Name,First Name,Points,Year,country,height,weight,age,event_count,avg_points,t-1,t-2,t-3,t-4,t-5,t-6,t-7
0,8372,1,Ondra,Adam,300.0,2019,CZE,185.0,67.0,26.0,3,100.000000,55.000000,80.000000,51.000000,65.428571,53.500000,90.000000,18.000000
1,8372,31,Ondra,Adam,55.0,2018,CZE,185.0,67.0,25.0,1,55.000000,80.000000,51.000000,65.428571,53.500000,90.000000,18.000000,56.666667
2,8372,28,Ondra,Adam,80.0,2017,CZE,185.0,67.0,24.0,1,80.000000,51.000000,65.428571,53.500000,90.000000,18.000000,56.666667,75.166667
3,8372,24,Ondra,Adam,51.0,2016,CZE,185.0,67.0,23.0,1,51.000000,65.428571,53.500000,90.000000,18.000000,56.666667,75.166667,NaN
4,8372,1,Ondra,Adam,458.0,2015,CZE,185.0,67.0,22.0,7,65.428571,53.500000,90.000000,18.000000,56.666667,75.166667,NaN,NaN
5,8372,3,Ondra,Adam,428.0,2014,CZE,185.0,67.0,21.0,8,53.500000,90.000000,18.000000,56.666667,75.166667,NaN,NaN,NaN
6,8372,11,Ondra,Adam,180.0,2013,CZE,185.0,67.0,20.0,2,90.000000,18.000000,56.666667,75.166667,NaN,NaN,NaN,NaN
7,8372,58,Ondra,Adam,18.0,2011,CZE,185.0,67.0,18.0,1,18.000000,56.666667,75.166667,NaN,NaN,NaN,NaN,NaN
8,8372,3,Ondra,Adam,340.0,2010,CZE,185.0,67.0,17.0,6,56.666667,75.166667,NaN,NaN,NaN,NaN,NaN,NaN
9,8372,1,Ondra,Adam,451.0,2009,CZE,185.0,67.0,16.0,6,75.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
sorted_merged_ml = merged_ml.sort_values(by=['Year', 'Points'], ascending=False)
sorted_merged_ml = sorted_merged_ml.reset_index(drop=True)
# sorted_merged_ml.to_pickle('../data/men_lead_no_drop.pkl')
sorted_merged_ml.to_csv('../data/men_lead_no_drop.csv', index=False)
sorted_merged_ml

,ID,Rank,Last Name,First Name,Points,Year,country,height,weight,age,event_count,avg_points,t-1,t-2,t-3,t-4,t-5,t-6,t-7
0,8372,1,Ondra,Adam,300.0,2019,CZE,185.0,67.0,26.0,3,100.000000,55.000000,80.000,51.000000,65.428571,53.500,90.000000,18.000
1,56609,2,Ginés López,Alberto,256.0,2019,ESP,NaN,NaN,17.0,6,42.666667,43.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,5089,3,McColl,Sean,206.0,2019,CAN,169.0,60.0,32.0,6,34.333333,25.666667,63.500,55.800000,50.600000,55.000,59.333333,59.375
3,14023,4,Harada,Kai,195.0,2019,JPN,NaN,NaN,20.0,5,39.000000,12.000000,6.500,NaN,NaN,NaN,NaN,NaN
4,8323,5,Ghisolfi,Stefano,190.0,2019,ITA,170.0,57.0,26.0,6,31.666667,66.571429,51.625,50.857143,36.000000,39.125,29.000000,34.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,798,73,Brette,Stéphane,3.0,1991,FRA,NaN,NaN,22.0,1,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,116,75,Bucher,Christoph,2.0,1991,GER,160.0,57.0,22.0,1,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,330,76,Florine,Hans,2.0,1991,USA,185.0,69.0,27.0,3,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,827,79,Kazuyoshi,Saleb,1.0,1991,JPN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
new_df = pd.read_csv('../data/men_lead_no_drop.csv')
new_df

,Unnamed: 0,ID,Rank,Last Name,First Name,Points,Year,country,height,weight,age,event_count,avg_points,t-1,t-2,t-3,t-4,t-5,t-6,t-7
0,0,8372,1,Ondra,Adam,300.0,2019,CZE,185.0,67.0,26.0,3,100.000000,55.000000,80.000,51.000000,65.428571,53.500,90.000000,18.000
1,1,56609,2,Ginés López,Alberto,256.0,2019,ESP,NaN,NaN,17.0,6,42.666667,43.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5089,3,McColl,Sean,206.0,2019,CAN,169.0,60.0,32.0,6,34.333333,25.666667,63.500,55.800000,50.600000,55.000,59.333333,59.375
3,3,14023,4,Harada,Kai,195.0,2019,JPN,NaN,NaN,20.0,5,39.000000,12.000000,6.500,NaN,NaN,NaN,NaN,NaN
4,4,8323,5,Ghisolfi,Stefano,190.0,2019,ITA,170.0,57.0,26.0,6,31.666667,66.571429,51.625,50.857143,36.000000,39.125,29.000000,34.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2011,798,73,Brette,Stéphane,3.0,1991,FRA,NaN,NaN,22.0,1,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,2012,116,75,Bucher,Christoph,2.0,1991,GER,160.0,57.0,22.0,1,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,2013,330,76,Florine,Hans,2.0,1991,USA,185.0,69.0,27.0,3,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,2014,827,79,Kazuyoshi,Saleb,1.0,1991,JPN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
